In [47]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
import datetime as dt
pymysql.install_as_MySQLdb()

In [48]:
# import sys
# sys.path.append('../../../../')
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [49]:
#Set up Salesforce
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [50]:
#Connect to your MySQL database
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

Object: Course
---

In [5]:
##Query course data from MySQL
course_data = pd.read_sql("SELECT * FROM course", conn)
course_data.head()

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [6]:
#Load course data into salesfore

for index, row in course_data.iterrows():

    record = {
        'CourseName__c': row['CourseName'],
        'CourseCode__c': row['CourseCode'],
        'CreditHours__c': row['CreditHours'],
        'BootCampCourse__c': row['BootCampCourse'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

Object: Class
---

In [51]:
##Query class data from MySQL
class_data = pd.read_sql("SELECT * FROM class", conn)
class_data.head()

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [52]:
#Query from salesforce to grab saleforce ID for course object
query = sf.query_all_iter('SELECT Name, CourseCode__c FROM Course__c')

salesforce_ids_list = []

for q in query:
    id_dict = {'salesforce_courseID': q['Name'], 'CourseCode':q['CourseCode__c']}
    salesforce_ids_list.append(id_dict)

salesforce_ids = pd.DataFrame(salesforce_ids_list)
salesforce_ids.head()

,salesforce_courseID,CourseCode
0,a033h000006VvKz,CIS-405
1,a033h000006VvLE,CIS-435
2,a033h000006VvKf,BC-DATAVIZ
3,a033h000006VvL4,CIS-438
4,a033h000006VvL9,CIS-430


In [53]:
#merge ID onto course table and then class table
course_merged = pd.merge(salesforce_ids, course_data)
class_course_merged = pd.merge(class_data, course_merged, how = 'left')
class_course_merged

,ID_Class,ID_Course,Section,StartDate,EndDate,salesforce_courseID,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,a033h000006VvKf,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,a033h000006VvKf,BC-DATAVIZ,Data Visualization and Analytics,12,1
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,a033h000006VvKk,BC-WEBDEV,Full Stack Web Development,12,1
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,a033h000006VvKp,BC-UIUX,User Interface/User Experience,12,1


In [10]:
#convert date columns to string
class_course_merged['StartDate'] = class_course_merged['StartDate'].astype(str)
class_course_merged['EndDate'] = class_course_merged['EndDate'].astype(str)
class_course_merged.dtypes

ID_Class                int64
ID_Course               int64
Section                object
StartDate              object
EndDate                object
salesforce_courseID    object
CourseCode             object
CourseName             object
CreditHours             int64
BootCampCourse          int64
dtype: object

In [11]:
#load class data into salesfore
for index, row in class_course_merged.iterrows():

    record = {
        'Cource_ID__c': row['salesforce_courseID'],
        'Section__c': row['Section'],
        'StartDate__c': row['StartDate'],
        'EndDate__c': row['EndDate']
    }
    
    try:
        sf.Class__c.create(record)
    except Exception as e:
        print(e)

Staff
---

In [54]:
#Query staff data from MySQL
staff_data = pd.read_sql("SELECT * FROM staff", conn)
staff_data.head()

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [13]:
#convert date columns to string
staff_data['BirthDate'] = staff_data['BirthDate'].astype(str)
staff_data.dtypes

ID_Staff       int64
EmployeeID    object
LastName      object
FirstName     object
MiddleName    object
BirthDate     object
dtype: object

In [14]:
#load staff data into salesfore
for index, row in staff_data.iterrows():

    record = {
        'EmployeeID__c': row['EmployeeID'],
        'LastName__c': row['LastName'],
        'FirstName__c': row['FirstName'],
        'MiddleName__c': row['MiddleName'],
        'BirthDate__c': row['BirthDate'],
    }
    
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)

Student
----

In [15]:
#Query student data from MySQL
student_data = pd.read_sql("SELECT * FROM student", conn)
student_data.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


In [16]:
#load student data into salesfore
for index, row in student_data.iterrows():

    record = {
        'Student_ID__c': row['StudentID'],
        'LastName__c': row['LastName'],
        'FirstName__c': row['FirstName'],
        'MiddleName__c': row['MiddleName'],
        'BirthDate__c': row['BirthDate'],
        'Gender__c': row['Gender'],
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

Class Participant
---

In [17]:
#Query classparticipant data from MySQL
classparticipant_data = pd.read_sql("SELECT * FROM classparticipant", conn)
classparticipant_data.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None


In [26]:
#Query from salesforce to grab saleforce ID for student object
query = sf.query_all_iter('SELECT Name, Student_ID__c FROM student__c')

salesforce_student_ids_list = []


for q in query:
    id_dict = {'Salesfore_StudentID': q['Name'], 'StudentID':q['Student_ID__c']}
    salesforce_student_ids_list.append(id_dict)

#convert studentID to string
salesforce_student_ids = pd.DataFrame(salesforce_student_ids_list)
salesforce_student_ids['StudentID'] = salesforce_student_ids['StudentID'].astype(int)
salesforce_student_ids['StudentID'] = salesforce_student_ids['StudentID'].astype(str)
salesforce_student_ids.head()


,Salesfore_StudentID,StudentID
0,a093h000000PI9f,25003936
1,a093h000000PI8r,25005602
2,a093h000000PI91,25006947
3,a093h000000PI9u,25006027
4,a093h000000PI8X,25002056


In [35]:
student_merged = pd.merge(student_data, salesforce_student_ids)
student_merged = student_merged[['ID_Student','StudentID', 'Salesfore_StudentID']]
student_merged.head()

,ID_Student,StudentID,Salesfore_StudentID
0,33,25004961,a093h000000PI7o
1,34,25003514,a093h000000PI7t
2,35,25005833,a093h000000PI7y
3,36,25002589,a093h000000PI83
4,37,25007185,a093h000000PI88


In [27]:
#Query from salesforce to grab saleforce ID for class object
query = sf.query_all_iter('SELECT Name, Section__c FROM class__c')

salesforce_class_ids_list = []


for q in query:
    id_dict = {'Salesfore_ClassID': q['Name'], 'Section':q['Section__c']}
    salesforce_class_ids_list.append(id_dict)

salesforce_class_ids = pd.DataFrame(salesforce_class_ids_list)
salesforce_class_ids.head()

,Salesfore_ClassID,Section
0,a063h0000025G03,GWU-ARL-DATA-PT-09-0
1,a063h0000025G0I,GWARL201905UIUX3
2,a063h0000025G08,GWDC201805DATA3
3,a063h0000025G0D,GWARL201905WEB3


In [39]:
class_merged = pd.merge(class_data, salesforce_class_ids)
class_merged = class_merged[['ID_Class','ID_Course', 'Salesfore_ClassID']]
class_merged.head()

,ID_Class,ID_Course,Salesfore_ClassID
0,1,1,a063h0000025G03
1,2,1,a063h0000025G08
2,3,2,a063h0000025G0D
3,4,3,a063h0000025G0I


In [42]:
##merge classparticipant (cp) with class and student tables
cp_student_merged = pd.merge(classparticipant_data, student_merged, how = 'left')
cp_student_class_merged = pd.merge(cp_student_merged, class_merged, how = 'left')

cp_student_class_merged.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate,StudentID,Salesfore_StudentID,ID_Course,Salesfore_ClassID
0,1,33,1,2020-03-16,None,25004961,a093h000000PI7o,1,a063h0000025G03
1,2,34,1,2020-03-16,None,25003514,a093h000000PI7t,1,a063h0000025G03
2,3,35,1,2020-03-16,None,25005833,a093h000000PI7y,1,a063h0000025G03
3,4,62,1,2020-03-16,None,25007334,a093h000000PIA9,1,a063h0000025G03
4,5,36,1,2020-03-16,None,25002589,a093h000000PI83,1,a063h0000025G03


In [43]:
#convert date columns to string
cp_student_class_merged['StartDate'] = cp_student_class_merged['StartDate'].astype(str)


In [45]:
for index, row in cp_student_class_merged.iterrows():

    record = {
        'Student_ID__c': row['Salesfore_StudentID'],
        'Class_ID__c': row['Salesfore_ClassID'],
        'StartDate__c': row['StartDate'],
        'EndDate__c': row['EndDate']
    }
    
    try:
        sf.classparticipant__c.create(record)
    except Exception as e:
        print(e)

Staff Assignments
---

In [55]:
staffassignment_data = pd.read_sql("SELECT * FROM staffassignment", conn)
staffassignment_data.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [58]:
#Query from salesforce to grab saleforce ID for student object
query = sf.query_all_iter('SELECT Name, EmployeeID__c FROM staff__c')

salesforce_staff_ids_list = []


for q in query:
    id_dict = {'Salesfore_StaffID': q['Name'], 'EmployeeID':q['EmployeeID__c']}
    salesforce_staff_ids_list.append(id_dict)

salesforce_staff_ids = pd.DataFrame(salesforce_staff_ids_list)

salesforce_staff_ids.head()

,Salesfore_StaffID,EmployeeID
0,a073h000002uAaH,000184220
1,a073h000002uAaR,000160655
2,a073h000002uAaM,000130109
3,a073h000002uAaW,000159108


In [67]:
##merge StaffAssignment (sa) with class and staff tables
staff_merged = pd.merge(salesforce_staff_ids, staff_data)
staffassignment_merged = pd.merge(staffassignment_data, staff_merged, how = 'left')
staffassignment_class_merged = pd.merge(staffassignment_merged, class_merged, how = 'left')

staffassignment_class_merged.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate,Salesfore_StaffID,EmployeeID,LastName,FirstName,MiddleName,BirthDate,ID_Course,Salesfore_ClassID
0,1,1,1,Teacher Assistant,2020-03-16,None,a073h000002uAaH,000184220,Wimberly,Sam,Nico,1995-07-05,1,a063h0000025G03
1,2,4,1,Teacher Assistant,2020-03-16,None,a073h000002uAaW,000159108,Popelka,Sarah,Nisan,1994-06-07,1,a063h0000025G03
2,3,3,1,Instructor,2020-03-16,None,a073h000002uAaR,000160655,Williams,Dartanion,De Angelo,1993-05-21,1,a063h0000025G03


In [68]:
staffassignment_class_merged['StartDate'] = staffassignment_class_merged['StartDate'].astype(str)
staffassignment_class_merged.dtypes

ID_StaffAssignment     int64
ID_Staff               int64
ID_Class               int64
Role                  object
StartDate             object
EndDate               object
Salesfore_StaffID     object
EmployeeID            object
LastName              object
FirstName             object
MiddleName            object
BirthDate             object
ID_Course              int64
Salesfore_ClassID     object
dtype: object

In [70]:
for index, row in staffassignment_class_merged.iterrows():

    record = {
        'Staff_ID__c': row['Salesfore_StaffID'],
        'Class_ID__c': row['Salesfore_ClassID'],
        'Role__c': row['Role'],
        'StartDate__c': row['StartDate'],
        'End_Date__c': row['EndDate']
    }
    
    try:
        sf.staffassignment__c.create(record)
    except Exception as e:
        print(e)